# Решение задачи №1 - Квантовый черный ящик - восстановление параметров квантового преобразования

**Решение команды №5:**   
Дмитрий Норкин,  
Екатерина Сажина,   
Сергей Бажин,  
Татьяна и Мария 

In [181]:
import numpy as np
from scipy.optimize import minimize
import json
# библиотека для рисования прогресс-бара
from tqdm import tqdm
import pickle

In [143]:
# создание произвольного нормального вектора размерности 4 -> далее
# из него через функцию generate_complex_vector получается вектор размерности 2 из комплексных чисел
def generateState():
    s0 = np.random.random_sample() * 2 - 1
    s1 = np.random.random_sample() * 2 - 1
    s2 = np.random.random_sample() * 2 - 1
    s3 = np.random.random_sample() * 2 - 1
    state = np.array([s0, s1, s2, s3]/np.sqrt(s0 ** 2 + s1 ** 2 + s2 ** 2 + s3 ** 2))
    return state

# функция для создания комплексного вектора из вещественной основы
def generate_complex_vector(state):
    return( state[0]+state[1]*1j,  state[2]+state[3]*1j )

generateState()

array([-0.63357551, -0.41477243,  0.08384084, -0.64770102])

In [144]:
# расчет теоритической вероятности на основе 
# пара комплексных чисел alpha , beta - входной вектор
# пара комплексных чисел m, n - базис измерения выходного состояния
# a, b, phi - параметры унитарной матрицы преобразования которую необходимо найти
# a, b - комплексные, phi - вещественное
def probability(a, b, phi, alpha, beta, m, n):
    part1 = np.abs(m) ** 2 * np.abs((a * alpha + b * beta)) ** 2
    part2 = np.conj(m) * n * (a*alpha + b*beta) * np.exp(-phi*1j) * (a*np.conj(beta) - alpha*np.conj(b))
    part3 = m * np.conj(n) * (np.conj(a)*np.conj(alpha) + np.conj(b)*np.conj(beta)) * np.exp(phi*1j) * (np.conj(a)*beta - np.conj(b)*alpha)
    part4 = np.abs(n) ** 2 * np.abs(np.conj(a)*beta - alpha*np.conj(b))
    return part1 + part2 + part3 + part4

In [146]:
# читаем пароль из файла формата {"password":"значение пароля"}
with open('password.json', 'r') as file:
    secret = json.load(file)
password = secret['password']

###  Задаём начальне параметры алгоритма

In [153]:
# индекс преобразования - целое число от 0 до 5, 
# 0-2 - тренировочные, 
# 3-5 - боевые на зачёт
process_index = 0

# число измерений итогового состояния
number_of_states = 100

# число входных кубитов (входных векторов для черного ящика)
NUMBER_OF_INPUT_STATES = 100

In [173]:
# функция расчёта отклонения теоритической вероятности (расчитанной через функцию probability) от 
# экспериментальной (измерена через обращение к функции run_measurement)
def loss_function(params, result):
    ar, ai, br , bi, phi = params
    N = len(result)
    mnk = 0
    for result_item in result:
        # получение величин измерения
        state = result_item[0]
        projector = result_item[1]
        p0_measured = result_item[2]
        
        # вычисление теоритической вероятности
        alpha, beta = generate_complex_vector(state) 
        m, n = generate_complex_vector(projector)
        a = ar + ai*1j
        b = br + bi*1j
        p0_theoretical = probability(a, b, phi, alpha, beta, m, n)
        mnk += pow((p0_theoretical - p0_measured),2)
    mnk = mnk/N
    return(mnk)

### отладочный вывод - проверим, что все функции реализованы корректно

In [149]:
# вещественная основа для входного вектора
state = generateState()
# вещественная основа для базиса в котором проводим измерение
projector = generateState()

# входной вектор (комплексные числа)
alpha, beta = generate_complex_vector(state) 

# комплексный базис измерения
m, n = generate_complex_vector(projector)

# Initial parameters
a = 1+1*1j
b = 0+1*1j
phi = 0

probability(a, b, phi, alpha, beta, m, n)

(1.7416306535453403+0.12014487245285516j)

In [150]:
# функция измерения состояния
def run_measurement(state, projector, process_index=0, number_of_states=100):
    """
    выполняет измерение состояния выходного кубита и возвращает пару чисел n0 и n1, где n0+n1 = number_of_states
    
    state - матрица из 4 чисел, представляющая собой вектор из комплексных элементов, входное состояние 
    преобразования
    
    projector - матрица из 4 чисел, представляющая собой вектор из комплексных элементов, базис измерения выходного
    состояния
    
    process_index - код преобразования для которого измеряется выходной вектор
    целое число от 0 до 5
    0-2 - тренировочные, 
    3-5 - боевые на зачёт
    number_of_states - число измеренных состояний выходного вектора
    """
    import socket, sys

    #request = password + ' 1 100 0.5 0.5 0.5 0.5 1 0 0 0'

    request = password + ' ' + str(process_index)+ ' ' + str(number_of_states) + ' ' + \
    ' '.join([str(elem) for elem in np.append(state, projector)])


    host = 'ape.qotlabs.org'
    port = 2018

    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.settimeout(2)

    try:
        s.connect((host, port))
    except:
        print('Unable to connect')
        sys.exit()

    s.send(request.encode())
    data = s.recv(4096).decode()
    n = [int(item) for item in data.split()]
    return(n[0], n[1])

In [151]:
run_measurement(state, projector)

(26, 74)

In [165]:
def perform_complete_measurement(process_index = 0, number_of_input_states = NUMBER_OF_INPUT_STATES):
    
    # список с итоговыми измерениями вероятности и входными данными
    result = []
    for i in tqdm(range(0,number_of_input_states)):
        # генерируем вход. состояние
        state = generateState()
        projector = generateState()
        n = run_measurement(state, projector)
        p0 = n[0] / number_of_states
        result.append((state, projector, p0))
    return (result)

In [167]:
%%time
# получаем результат измерений для нулевого преобразования
result0 = perform_complete_measurement(process_index = 0)   

100%|██████████| 100/100 [00:01<00:00, 65.13it/s]

CPU times: user 116 ms, sys: 42.6 ms, total: 158 ms
Wall time: 1.54 s


In [168]:
%%time
# получаем результат измерений для нулевого преобразования
result1 = perform_complete_measurement(process_index = 1)   

100%|██████████| 100/100 [00:01<00:00, 77.76it/s]

CPU times: user 132 ms, sys: 30.2 ms, total: 162 ms
Wall time: 1.29 s


In [169]:
%%time
# получаем результат измерений для нулевого преобразования
result2 = perform_complete_measurement(process_index = 2)   

100%|██████████| 100/100 [00:01<00:00, 79.18it/s]

CPU times: user 114 ms, sys: 36.7 ms, total: 151 ms
Wall time: 1.27 s


In [170]:
result0[0][0], result0[0][1], result0[0][2]

(array([ 0.62006238, -0.04210918,  0.23026873, -0.74881625]),
 array([-0.21209376,  0.40199031,  0.63027525, -0.62942286]),
 0.53)

###  Находим параметры преобразования через оптимизацию функции потерь

In [176]:
# начальная точка
x0 = [0.5, 0.5, 0.5, 0.5, 1]
# доп. параметры для функции потерь - наши данные о входном векторе, базисе измерения и измеренной вероятности
args = (result0)
optimization_result = minimize(loss_function, x0, args=args, method='L-BFGS-B')
optimization_result

/usr/lib/python3.6/site-packages/scipy/optimize/optimize.py:643: ComplexWarning: Casting complex values to real discards the imaginary part
  grad[k] = (f(*((xk + d,) + args)) - f0) / d[k]
/usr/lib/python3.6/site-packages/scipy/optimize/lbfgsb.py:328: ComplexWarning: Casting complex values to real discards the imaginary part
  isave, dsave, maxls)


      fun: (0.007198317094138418-1.462700411771958e-05j)
 hess_inv: <5x5 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 6.19894761e-06, -8.67301023e-06, -2.81632356e-07, -2.36338726e-06,
        1.14387666e-06])
  message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
     nfev: 450
      nit: 59
   status: 0
  success: True
        x: array([ 0.6864223 , -0.62495438,  0.00660917,  0.05271808, -1.46859191])

In [187]:
# получение итоговой матрицы преобразования
def get_black_box_matrix(optimization_result):
    a = optimization_result.x[0] + optimization_result.x[1]*1j
    b = optimization_result.x[2] + optimization_result.x[3]*1j
    phi = optimization_result.x[4]
    
    black_box = np.array([[a , b],
                          [-np.exp(1j*phi)*np.conj(b), np.exp(1j*phi)*np.conj(a)]])
    
    return(black_box)

###  Это итоговые данные по оптимизации преобразвания 0

In [182]:
[round(item, 7) for item in optimization_result.x]

[0.6862669, -0.6251177, 0.0066188, 0.052728, -1.4690923]

In [183]:
a = optimization_result.x[0] + optimization_result.x[1]*1j
b = optimization_result.x[2] + optimization_result.x[3]*1j
phi = optimization_result.x[4]
a, b, phi

((0.6862669057603956-0.6251177298046051j),
 (0.006618795194369992+0.05272801551057072j),
 -1.4690923203311566)

### Получение ответов на боевых данных
Поскольку на боевых данных ответ можно получить только 1 раз - они сохранены в файлы dump3,4,5

###  5-ый ящик

In [192]:
# загружаем данные
with open('dump5.pkl','rb') as file:
    result5 = pickle.load(file)

In [185]:
x0 = [0.1, 0.3, 0.4, 0.2, 1]
args = (result5)
optimization_result = minimize(loss_function, x0, args=args, method='L-BFGS-B')
optimization_result

/usr/lib/python3.6/site-packages/scipy/optimize/optimize.py:643: ComplexWarning: Casting complex values to real discards the imaginary part
  grad[k] = (f(*((xk + d,) + args)) - f0) / d[k]
/usr/lib/python3.6/site-packages/scipy/optimize/lbfgsb.py:328: ComplexWarning: Casting complex values to real discards the imaginary part
  isave, dsave, maxls)


      fun: (0.0003443266298777796-0.0017071250261712196j)
 hess_inv: <5x5 LbfgsInvHessProduct with dtype=float64>
      jac: array([-1.14142094e-06,  1.44417898e-06, -9.00175117e-07,  2.01986865e-08,
       -6.99451348e-07])
  message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
     nfev: 234
      nit: 32
   status: 0
  success: True
        x: array([ 0.51008814, -0.42767797,  0.46589494,  0.50171197,  2.33126327])

In [191]:
get_black_box_matrix(optimization_result)

array([[ 0.51008814-0.42767797j,  0.46589494+0.50171197j],
       [-0.04237481-0.68335743j, -0.66144205+0.07478488j]])

###  Для 4-ого черного ящика

In [193]:
# загружаем данные
with open('dump4.pkl','rb') as file:
    result4 = pickle.load(file)

In [194]:
x0 = [0.1, 0.3, 0.4, 0.2, 1]
args = (result4)
optimization_result = minimize(loss_function, x0, args=args, method='L-BFGS-B')
optimization_result

/usr/lib/python3.6/site-packages/scipy/optimize/optimize.py:643: ComplexWarning: Casting complex values to real discards the imaginary part
  grad[k] = (f(*((xk + d,) + args)) - f0) / d[k]
/usr/lib/python3.6/site-packages/scipy/optimize/lbfgsb.py:328: ComplexWarning: Casting complex values to real discards the imaginary part
  isave, dsave, maxls)


      fun: (-0.017831400523716127+0.015167070879511718j)
 hess_inv: <5x5 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 5.26419186e-06,  1.25940924e-07, -2.85882429e-07, -1.42073853e-06,
        6.58847976e-07])
  message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
     nfev: 108
      nit: 15
   status: 0
  success: True
        x: array([ 0.21289382,  0.05285397,  1.00178535, -0.37329921,  0.21362004])

In [195]:
get_black_box_matrix(optimization_result)

array([[ 0.21289382+0.05285397j,  1.00178535-0.37329921j],
       [-0.89987557-0.5771916 j,  0.21925971-0.00651931j]])

###  Для 3-его черного ящика

In [196]:
# загружаем данные
with open('dump3.pkl','rb') as file:
    result3 = pickle.load(file)

In [197]:
x0 = [0.1, 0.3, 0.4, 0.2, 1]
args = (result3)
optimization_result = minimize(loss_function, x0, args=args, method='L-BFGS-B')
optimization_result

/usr/lib/python3.6/site-packages/scipy/optimize/optimize.py:643: ComplexWarning: Casting complex values to real discards the imaginary part
  grad[k] = (f(*((xk + d,) + args)) - f0) / d[k]
/usr/lib/python3.6/site-packages/scipy/optimize/lbfgsb.py:328: ComplexWarning: Casting complex values to real discards the imaginary part
  isave, dsave, maxls)


      fun: (0.004131297049801436+0.000990659211232728j)
 hess_inv: <5x5 LbfgsInvHessProduct with dtype=float64>
      jac: array([-6.30511268e-06,  6.44475792e-06, -6.56662225e-06,  6.68215483e-07,
        3.56572410e-07])
  message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
     nfev: 132
      nit: 19
   status: 0
  success: True
        x: array([-0.68218174,  0.50257604,  0.32452351,  0.3180893 ,  1.42782481])

In [198]:
get_black_box_matrix(optimization_result)

array([[-0.68218174+0.50257604j,  0.32452351+0.3180893 j],
       [-0.36108354-0.27588945j,  0.40024762-0.74683094j]])